# Notebook for lollipops

## Housekeeping

### Import dependencies

In [ ]:
import pandas as pd
import numpy as np

import chickenstats

from chickenstats.chicken_nhl import Season, Scraper
from chickenstats.chicken_nhl.helpers import norm_coords
from chickenstats.chicken_nhl.info import NHL_COLORS, team_codes
import chickenstats.utilities
from chickenstats.api import ChickenStats, ChickenUser, ChickenToken

from hockey_rink import NHLRink
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import matplotlib.patheffects as mpe
import matplotlib.ticker as ticker
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path

import datetime as dt

import math

from typing import Optional

### Pandas options

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### Environment variables

In [ ]:
env_path = Path("../.env")
load_dotenv(env_path)

### Chickenstats matplotlib style

In [ ]:
plt.style.use("chickenstats")

## API instance

### Connect to API

In [ ]:
api_instance = ChickenStats(api_url="http://macstudio.local")

## Scrape data

### Schedule

In [ ]:
season = Season(2024)

In [ ]:
schedule = season.schedule()

### Standings and team names

In [ ]:
standings = season.standings

In [ ]:
team_names = standings.sort_values(by="team_name").team_name.str.upper().tolist()
team_codes = standings.sort_values(by="team_name").team.str.upper().tolist()
team_names_dict = dict(zip(team_codes, team_names))

### Game IDs

In [ ]:
api_game_ids = api_instance.check_pbp_game_ids(season=[20242025])

In [ ]:
conds = np.logical_and(
    schedule.game_state == "OFF", ~schedule.game_id.isin(api_game_ids)
)

game_ids = schedule.loc[conds].game_id.unique().tolist()

### Play-by-play

In [ ]:
scraper = Scraper(game_ids)

In [ ]:
pbp = scraper.play_by_play

### Stats

In [ ]:
scraper.prep_stats(level="period", score=True, teammates=True, opposition=True)
stats = scraper.stats

### API IDs

In [ ]:
conds = np.logical_and(
    schedule.game_state == "OFF",
    np.logical_or(schedule.home_team == "NSH", schedule.away_team == "NSH"),
)

nsh_ids = schedule.loc[conds].game_id.unique().tolist()

In [ ]:
api_pbp = api_instance.download_pbp(game_id=nsh_ids)

In [ ]:
seasons = [int(f"{x}{x+1}") for x in range(2010, 2025)]

strengths = ["5v5", "3v3", "4v4", "5v4", "5v3", "4v3"]


stamkos_stats = api_instance.download_game_stats(
    season=seasons, player="STEVEN STAMKOS", strength_state=strengths
)

In [ ]:
scraper = Scraper(2024020369)

In [ ]:
pbp = scraper.play_by_play

In [ ]:
stats = scraper.stats

In [ ]:
stats.sort_values(by="icf_p60", ascending=False)

In [ ]:
game_ids_pbp_api = api_instance.check_pbp_game_ids(season=[20242025])

In [ ]:
game_ids_stats_api = api_instance.check_stats_game_ids(season=[20242025])

In [ ]:
[x for x in game_ids_pbp_api if x not in game_ids_stats_api]

In [ ]:
conds = np.logical_and(pbp.game_id.isin(game_ids_api), ~pbp.id.isin(play_ids_api))

# api_instance.upload_pbp(pbp.loc[conds].copy(deep=True))

pbp.loc[conds]

In [ ]:
game_ids_api[-1]

In [ ]:
api_instance = ChickenStats(api_url="http://macstudio.local")

In [ ]:
test_stuff = api_instance.download_pbp(game_id=2023020001)

In [ ]:
test_stuff

In [ ]:
api_instance.upload_stats(stats)

In [ ]:
player = ["JUUSE SAROS", "FILIP FORSBERG", "MATT DUCHENE"]
strengths = ["5v5"]
# strengths=None


stats = api_instance.download_game_stats(player=player, strength_state=strengths)

In [ ]:
stats

In [ ]:
player = "JUUSE.SAROS"
strengths = ["5v5"]

conds = np.logical_and(stats.eh_id == player, stats.strength_state.isin(strengths))

stats.loc[conds].sort_values(by="toi", ascending=False)

In [ ]:
pbp.loc[pbp.game_id == 2024020028].head(100)

In [ ]:
api_instance = ChickenStats(api_url="http://macstudio.local")

In [ ]:
test = api_instance.download_game_stats(
    player="JUUSE SAROS", season=years, strength_state="5v5"
)

In [ ]:
game_ids = [2024020001, 2024020002]

scraper = Scraper(game_ids)

In [ ]:
pbp = scraper.play_by_play

In [ ]:
scraper.prep_stats(level="period", score=True, teammates=True, opposition=True)
stats = scraper.stats

In [ ]:
api_instance.upload_stats(stats)

In [ ]:
help(scraper.prep_stats)